In [1]:
######## 실행

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !git clone https://github.com/ultralytics/ultralytics.git

Cloning into 'ultralytics'...
remote: Enumerating objects: 37794, done.
remote: Counting objects: 100% (199/199), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 37794 (delta 109), reused 132 (delta 69), pack-reused 37595 (from 1)
Receiving objects: 100% (37794/37794), 29.76 MiB | 14.15 MiB/s, done.
Resolving deltas: 100% (27704/27704), done.


In [2]:
######## 실행

cd /content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/

/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics


In [3]:
pwd

'/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics'

In [6]:
######## 실행

!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.1/871.1 kB 53.1 MB/s eta 0:00:00


# YOLOb8을 사용하려면
- train, val, test 폴더 안에 각각 images, labels 파일이 있어야함
- images, labels 안에는 이름이 같지만 형식이 다른 파일이 있어야한다. (예, images안에 people_img.jpg labels안에 people_img.txt)
- data.yaml 파일 안에는 클래스 개수, 클래스명, 모든 이미지파일의 정보가 담겨있어야한다.
- train이미지경로, val이미지경로, test이미지경로의 정보를 담은 train.txt, val.txt, test.txt 파일을 만든다. data.yaml 파일에 train.txt, val.txt, test.txt 경로를 넣는다.

In [8]:
######## 실행

import os
import json
from pathlib import Path
from PIL import Image
import shutil
from glob import glob
import yaml
from ultralytics import YOLO

WARNING ⚠️ Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.yaml'
Update settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [27]:
# # 경로 설정
# train_image_base = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/img/train'
# val_image_base = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/img/val'
# test_image_base = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/test/image'

# train_label_base = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/label/train'
# val_label_base = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/label/val'
# test_label_base = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/test/label'

# output_dir = '/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics'
# emotion_dirs = ['anger', 'happy', 'panic', 'sadness']

# def convert_to_yolo_format(box_data, img_width, img_height):
#     '''
#     JSON에서 추출한 박스 좌표를 YOLO 형식으로 변환합니다.
#     YOLO 포맷은 좌표를 이미지 크기에 대해 정규화합니다.
#     '''
#     min_x = box_data['minX']
#     min_y = box_data['minY']
#     max_x = box_data['maxX']
#     max_y = box_data['maxY']

#     x_center = (min_x + max_x) / 2 / img_width
#     y_center = (min_y + max_y) / 2 / img_height
#     width = (max_x - min_x) / img_width
#     height = (max_y - min_y) / img_height

#     return f"0 {x_center} {y_center} {width} {height}"

# # 디렉토리 생성
# os.makedirs(os.path.join(output_dir, 'train', 'images'), exist_ok=True)
# os.makedirs(os.path.join(output_dir, 'train', 'labels'), exist_ok=True)
# os.makedirs(os.path.join(output_dir, 'val', 'images'), exist_ok=True)
# os.makedirs(os.path.join(output_dir, 'val', 'labels'), exist_ok=True)
# os.makedirs(os.path.join(output_dir, 'test', 'images'), exist_ok=True)
# os.makedirs(os.path.join(output_dir, 'test', 'labels'), exist_ok=True)

# # 데이터 복사 및 라벨 처리
# for emotion in emotion_dirs:
#     # 이미지 및 라벨 파일 경로 설정
#     train_images_dir = os.path.join(train_image_base, emotion)
#     val_images_dir = os.path.join(val_image_base, emotion)
#     test_images_dir = os.path.join(test_image_base, emotion)

#     train_label_file = os.path.join(train_label_base, f"train_{emotion}.json")
#     val_label_file = os.path.join(val_label_base, f"val_{emotion}.json")
#     test_label_file = os.path.join(test_label_base, f"test_{emotion}.json")

#     # 이미지 파일 리스트 가져오기
#     train_images = [f for f in os.listdir(train_images_dir) if f.endswith('.jpg')]
#     val_images = [f for f in os.listdir(val_images_dir) if f.endswith('.jpg')]
#     test_images = [f for f in os.listdir(test_images_dir) if f.endswith('.jpg')]

#     # 라벨 데이터 읽기
#     with open(train_label_file, 'r', encoding='EUC-KR') as f:
#         train_labels = json.load(f)
#     with open(val_label_file, 'r', encoding='EUC-KR') as f:
#         val_labels = json.load(f)
#     with open(test_label_file, 'r', encoding='EUC-KR') as f:
#         test_labels = json.load(f)

#     # 훈련 데이터 복사 및 라벨 파일 생성
#     for img_file in train_images:
#         img_path = os.path.join(train_images_dir, img_file)
#         dest_img_path = os.path.join(output_dir, 'train', 'images', img_file)

#         # 이미지 복사
#         shutil.copy(img_path, dest_img_path)

#         # 이미지 크기 가져오기
#         with Image.open(dest_img_path) as img:
#             img_width, img_height = img.size

#         img_label_data = None
#         for item in train_labels:
#             if item["filename"] == img_file:
#                 img_label_data = item["annot_B"]["boxes"]
#                 break

#         if img_label_data:
#             yolo_box = convert_to_yolo_format(img_label_data, img_width, img_height)

#             # 라벨 파일로 저장
#             label_file = os.path.splitext(img_file)[0] + '.txt'
#             label_path = os.path.join(output_dir, 'train', 'labels', label_file)
#             with open(label_path, 'w') as lf:
#                 lf.write(yolo_box + '\n')

#     # 검증 데이터 복사 및 라벨 파일 생성
#     for img_file in val_images:
#         img_path = os.path.join(val_images_dir, img_file)
#         dest_img_path = os.path.join(output_dir, 'val', 'images', img_file)

#         # 이미지 복사
#         shutil.copy(img_path, dest_img_path)

#         # 이미지 크기 가져오기
#         with Image.open(dest_img_path) as img:
#             img_width, img_height = img.size

#         img_label_data = None
#         for item in val_labels:
#             if item["filename"] == img_file:
#                 img_label_data = item["annot_B"]["boxes"]
#                 break

#         if img_label_data:
#             yolo_box = convert_to_yolo_format(img_label_data, img_width, img_height)

#             # 라벨 파일로 저장
#             label_file = os.path.splitext(img_file)[0] + '.txt'
#             label_path = os.path.join(output_dir, 'val', 'labels', label_file)
#             with open(label_path, 'w') as lf:
#                 lf.write(yolo_box + '\n')

#     # 테스트 데이터 복사 및 라벨 파일 생성
#     for img_file in test_images:
#         img_path = os.path.join(test_images_dir, img_file)
#         dest_img_path = os.path.join(output_dir, 'test', 'images', img_file)

#         # 이미지 복사
#         shutil.copy(img_path, dest_img_path)

#         # 이미지 크기 가져오기
#         with Image.open(dest_img_path) as img:
#             img_width, img_height = img.size

#         img_label_data = None
#         for item in test_labels:
#             if item["filename"] == img_file:
#                 img_label_data = item["annot_B"]["boxes"]
#                 break

#         if img_label_data:
#             yolo_box = convert_to_yolo_format(img_label_data, img_width, img_height)

#             # 라벨 파일로 저장
#             label_file = os.path.splitext(img_file)[0] + '.txt'
#             label_path = os.path.join(output_dir, 'test', 'labels', label_file)
#             with open(label_path, 'w') as lf:
#                 lf.write(yolo_box + '\n')


In [32]:
!pwd

train_img_list = glob('/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/train/images/*.jpg')
train_txt_list = glob('/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/train/labels/*.txt')
val_img_list = glob('/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/val/images/*.jpg')
val_txt_list = glob('/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/val/labels/*.txt')
test_img_list = glob('/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/test/images/*.jpg')
test_txt_list = glob('/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/test/labels/*.txt')

print(len(train_img_list), len(train_txt_list))
print(len(val_img_list), len(val_txt_list))
print(len(test_img_list), len(test_txt_list))

/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics
5891 5889
1166 1166
1170 1170


In [34]:
# 이미지와 라벨 파일 경로
train_img_list = glob('/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/train/images/*.jpg')
train_txt_list = glob('/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/train/labels/*.txt')

# 이미지 파일 이름만 추출
train_img_names = {os.path.splitext(os.path.basename(img))[0] for img in train_img_list}
# 라벨 파일 이름만 추출
train_txt_names = {os.path.splitext(os.path.basename(txt))[0] for txt in train_txt_list}

# 라벨 파일이 없는 이미지 파일 찾기
missing_labels = train_img_names - train_txt_names
# 불필요한 라벨 파일 찾기
extra_labels = train_txt_names - train_img_names

print("Missing labels:", len(missing_labels))
print("Extra labels:", len(extra_labels))

# 누락된 라벨 파일 삭제
for img_name in missing_labels:
    label_file = os.path.join('/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/train/labels', f"{img_name}.txt")
    if os.path.exists(label_file):
        os.remove(label_file)
        print(f"Removed missing label: {label_file}")

# 불필요한 라벨 파일 삭제
for txt_name in extra_labels:
    label_file = os.path.join('/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/train/labels', f"{txt_name}.txt")
    if os.path.exists(label_file):
        os.remove(label_file)
        print(f"Removed extra label: {label_file}")

Missing labels: 2
Extra labels: 0


In [36]:
# with open('/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/train.txt', 'w') as f:
#   f.write('\n'.join(train_img_list) + '\n')

# with open('/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/val.txt', 'w') as f:
#   f.write('\n'.join(val_img_list) + '\n')

### 생각해보니 test는 안만들어도 될듯..?
# with open('/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/test.txt', 'w') as f:
#   f.write('\n'.join(test_img_list) + '\n')

In [41]:
# # yaml 파일 만들기

# with open('/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/data.yaml', 'r') as f:
#   data = yaml.load(f, Loader=yaml.FullLoader)

# print(data)

# data['train'] = '/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/train.txt'
# data['val'] = '/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/val.txt'
# data['test'] = '/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/test.txt'

# # yaml파일에 경로 적기
# with open('/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/data.yaml', 'w') as f:
#   yaml.dump(data, f)

# # nc는 number of class : 클래스개수
# # name 에는 클래스 이름
# # 중요한건 경로를 넣어주는 것

{'names': ['face'], 'nc': 1, 'train': '/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/train.txt', 'val': '/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/val.txt'}


In [7]:
######## 실행

# yolov8n 가장 가벼운 나노모델
!yolo detect train data=data.yaml model=yolov8n.pt epochs=10 lr0=0.01

/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics
Ultralytics YOLOv8.2.82 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_con

In [9]:
# test 데이터 모델 예측 및 시각화
!yolo detect predict data=data.yaml model=yolov8n.pt source=/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/test/images

WARNING ⚠️ Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.yaml'
Update settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics YOLOv8.2.82 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLOv8n summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

image 1/1170 /content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/test/images/00riaf6eb09fb4924b3a7f4b37d3af4870b733b5bfaa8977603fafff94104n4p8.jpg: 480x640 1 person, 1 tie, 53.1ms
image 2/1170 /content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/test/images/01h6ce85f03f4589312446c62372f5cb24ea3083e41d2b1f2ec95393e037fpv2i.jpg: 480x640 1 person, 7.7ms
image 3/1170 /content/drive

In [11]:
# 모델 로드
model = YOLO('runs/detect/train2/weights/best.pt')

# 테스트 이미지 경로
test_image_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/minji.jpg'

# 예측 수행
results = model.predict(test_image_path)

# 첫 번째 결과 선택
result = results[0]

# 결과 출력
result.show()  # 결과를 이미지로 표시

# 결과 저장
result.save('/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/minji_result.jpg')  # 결과 저장 경로 지정



image 1/1 /content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/minji.jpg: 640x480 1 face, 8.9ms
Speed: 2.2ms preprocess, 8.9ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 480)


'/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/minji_result.jpg'

In [12]:
# 모델 성능 평가
!yolo detect val data=data.yaml model=yolov8n.pt

Ultralytics YOLOv8.2.82 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLOv8n summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
val: Scanning /content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/val/labels.cache... 1164 images, 0 backgrounds, 5 corrupt: 100% 1166/1166 [00:00<?, ?it/s]
val: WARNING ⚠️ /content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/val/images/00ll57ed4d58f3e6c766e65168fc704d9e43ca0b1de8dba8927517b474f083s8x.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/val/images/03azd3bd258ed2d883a62ad1bde1a583aa5dd3bb9a3e738d7db0a1d2931e86tx5.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/val/images/05bm462b58f3e2803222f677525c32f112fd1c5b753346a01f228e9b026657qhq.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/MyDrive/project/KDT

In [13]:
# 모델 가중치 저장
!ls /content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics

CITATION.cff	 docker    LICENSE	   README.md	    test      train	   val
CONTRIBUTING.md  docs	   mkdocs.yml	   README.zh-CN.md  tests     train.txt    val.txt
data.yaml	 examples  pyproject.toml  runs		    test.txt  ultralytics  yolov8n.pt


In [18]:
# 모델 zip파일로 저장
# 저장되는 것은 다음과 같다
# yolov8n_trained.pt (훈련된 모델 가중치 파일)
# data.yaml (데이터셋 정보 파일)

!zip -r model.zip /content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/runs/detect/train2/weights/best.pt /content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/runs/detect/train2/weights/last.pt /content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/data.yaml

  adding: content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/runs/detect/train2/weights/best.pt (deflated 9%)
  adding: content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/runs/detect/train2/weights/last.pt (deflated 9%)
  adding: content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/data.yaml (deflated 59%)


In [ ]:
# ### 배포한 모델을 간단하게 사용하는 예시

# import torch
# from yolov8 import YOLOv8

# # 모델 로드
# model = YOLOv8.load('yolov8n_trained.pt')

# # 예측 수행
# image_path = 'path/to/your/image.jpg'
# results = model.predict(image_path)

# # 결과 출력
# print(results)

In [15]:
cd runs/detect/train2/weights

/content/drive/MyDrive/project/KDT_kiosk/code/extracode/ultralytics/runs/detect/train2/weights


In [16]:
ls

best.pt  last.pt
